<a href="https://colab.research.google.com/github/alecbidaran/Pytorch_excersies/blob/main/torch_knowledge_distilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch_snippets pytorch_model_summary


     |████████████████████████████████| 41 kB 366 kB/s 
     |████████████████████████████████| 10.9 MB 8.9 MB/s 
     |████████████████████████████████| 216 kB 59.4 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 56 kB 6.0 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 


In [ ]:
from torch_snippets import *
from torchvision import transforms,datasets
from sklearn.model_selection import train_test_split
device = 'cuda' 
import cv2
import matplotlib.pyplot as plt 
import glob
import torch.nn.functional as F

In [ ]:
class Teacher_network(nn.Module): 
  def __init__(self,in_channels=1,num_class=10):
    super(Teacher_network,self).__init__()
    self.num_class=num_class
    self.in_channels=in_channels
    model=[nn.Conv2d(self.in_channels,64,3,stride=1,padding=1),nn.InstanceNorm2d(64),nn.ReLU(),nn.MaxPool2d(2,2)]
    model+=[nn.Conv2d(64,128,3,stride=1,padding=1),nn.InstanceNorm2d(64),nn.ReLU(),nn.MaxPool2d(2,2)]
    self.model=nn.Sequential(*model)
    self.flatten=nn.Flatten()
    self.fc=nn.Linear(128*7*7,10)
  def forward(self,x):
    x=self.model(x)
    f=self.flatten(x)
    y=self.fc(f)
    return y
class student_network(nn.Module): 
  def __init__(self,in_channels=1,num_class=10):
    super(student_network,self).__init__()
    self.num_class=num_class
    self.in_channels=in_channels
    model=[nn.Conv2d(self.in_channels,8,3,stride=1,padding=1),nn.InstanceNorm2d(8),nn.ReLU(),nn.MaxPool2d(2,2)]
    model+=[nn.Conv2d(8,64,3,stride=1,padding=1),nn.InstanceNorm2d(64),nn.ReLU(),nn.MaxPool2d(2,2)]
    self.model=nn.Sequential(*model)
    self.flatten=nn.Flatten()
    self.fc=nn.Linear(64*7*7,10)
  def forward(self,x):
    x=self.model(x)
    f=self.flatten(x)
    y=self.fc(f)
    return y

In [ ]:
teacher=Teacher_network(1,10).to(device)


In [ ]:
from torchsummary import summary

In [ ]:
summary(teacher,(1,28,28))
summary(student,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
    InstanceNorm2d-2           [-1, 64, 28, 28]               0
              ReLU-3           [-1, 64, 28, 28]               0
         MaxPool2d-4           [-1, 64, 14, 14]               0
            Conv2d-5          [-1, 128, 14, 14]          73,856
    InstanceNorm2d-6          [-1, 128, 14, 14]               0
              ReLU-7          [-1, 128, 14, 14]               0
         MaxPool2d-8            [-1, 128, 7, 7]               0
           Flatten-9                 [-1, 6272]               0
           Linear-10                   [-1, 10]          62,730
Total params: 137,226
Trainable params: 137,226
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.91
Params size (MB): 0.52
Estimated T

# Dataset

In [ ]:
train_dataset=datasets.MNIST("./",download=True,train=True,transform=transforms.ToTensor())
valid_datasets=datasets.MNIST("./",download=True,train=False,transform=transforms.ToTensor())

In [ ]:
train_dl=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dl=DataLoader(valid_datasets,batch_size=32,shuffle=False)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(teacher.parameters(),lr=1e-4)

In [ ]:
def train_batch(batch):
  data,label=batch
  teacher.train()
  optimizer.zero_grad()
  pred=teacher(data.cuda())
  loss=criterion(pred,label.cuda())
  acc=(torch.max(pred,1)[1]==label.cuda()).float().mean()
  loss.backward()
  optimizer.step()
  return loss.item(),acc.item()
@torch.no_grad()
def valid_batch(batch):
  data,label=batch
  teacher.eval()
  pred=teacher(data.cuda())
  loss=criterion(pred,label.cuda())
  acc=(torch.max(pred,1)[1]==label.cuda()).float().mean()
  return loss.item(),acc.item()

In [ ]:
n_epoch=10
log=Report(n_epoch)
for epochs in range(n_epoch):
  N = len(train_dl)
  for i,data in enumerate(train_dl):
    loss,acc=train_batch(data)
    log.record(epochs+(i+1)/N,trn_loss=loss,trn_acc=acc,end='\r')
  N = len(val_dl)
  for i,batch in enumerate(val_dl):
    loss,acc=valid_batch(batch)
    log.record(epochs+(i+1)/N,val_loss=loss,val_acc=acc,end='\r')
 

  log.report_avgs(epochs+1)

EPOCH: 1.000	trn_loss: 0.157	trn_acc: 0.957	val_loss: 0.065	val_acc: 0.980	(12.71s - 114.43s remaining)
EPOCH: 2.000	trn_loss: 0.051	trn_acc: 0.985	val_loss: 0.042	val_acc: 0.987	(25.34s - 101.35s remaining)
EPOCH: 3.000	trn_loss: 0.037	trn_acc: 0.989	val_loss: 0.043	val_acc: 0.987	(38.03s - 88.74s remaining)
EPOCH: 4.000	trn_loss: 0.029	trn_acc: 0.992	val_loss: 0.031	val_acc: 0.988	(50.62s - 75.93s remaining)
EPOCH: 5.000	trn_loss: 0.023	trn_acc: 0.993	val_loss: 0.035	val_acc: 0.989	(63.40s - 63.40s remaining)
EPOCH: 6.000	trn_loss: 0.019	trn_acc: 0.994	val_loss: 0.032	val_acc: 0.989	(76.04s - 50.70s remaining)
EPOCH: 7.000	trn_loss: 0.015	trn_acc: 0.996	val_loss: 0.034	val_acc: 0.988	(88.60s - 37.97s remaining)
EPOCH: 8.000	trn_loss: 0.012	trn_acc: 0.996	val_loss: 0.030	val_acc: 0.991	(101.30s - 25.32s remaining)
EPOCH: 9.000	trn_loss: 0.009	trn_acc: 0.998	val_loss: 0.040	val_acc: 0.987	(114.07s - 12.67s remaining)
EPOCH: 10.000	trn_loss: 0.009	trn_acc: 0.998	val_loss: 0.036	val_acc:

In [ ]:
student=student_network(1,10).to(device)
student_loss=nn.CrossEntropyLoss()
student_opt=torch.optim.Adam(student.parameters(),lr=1e-4)
kd_loss=nn.KLDivLoss(reduction="batchmean")

In [ ]:
def student_train(batch,temp=7,alpha=0.3):
  data,label=batch
  teacher.eval()
  student.train()
  student_opt.zero_grad()
  teacher_pred=teacher(data.cuda())
  student_pred=student(data.cuda())
  student_loss=criterion(student_pred,label.cuda())
  distil_loss=kd_loss(F.softmax(student_pred,dim=1)/temp,F.softmax(teacher_pred,dim=1)/temp)
  loss=alpha*student_loss+(1-alpha)*distil_loss
  acc=(torch.max(student_pred,1)[1]==label.cuda()).float().mean()
  loss.backward()
  student_opt.step()
  
  return loss.item(),acc.item()
@torch.no_grad()
def student_eval(batch,temp=7,alpha=0.3):
  data,label=batch
  teacher.eval()
  student.eval()
  student_opt.zero_grad()
  teacher_pred=teacher(data.cuda())
  student_pred=student(data.cuda())
  student_loss=criterion(student_pred,label.cuda())
  distil_loss=kd_loss(F.softmax(student_pred,dim=1)/temp,F.softmax(teacher_pred,dim=1)/temp)
  loss=student_loss*alpha+(1-alpha)*distil_loss
  acc=(torch.max(student_pred,1)[1]==label.cuda()).float().mean()
  return loss.item(),acc.item()

In [ ]:
n_epoch=10
log=Report(n_epoch)
for epochs in range(n_epoch):
  N = len(train_dl)
  for i,data in enumerate(train_dl):
    loss,acc=student_train(data,temp=7,alpha=0.3)
    log.record(epochs+(i+1)/N,trn_loss=loss,trn_acc=acc,end='\r')
  N = len(val_dl)
  for i,batch in enumerate(val_dl):
    loss,acc=student_eval(batch,temp=7,alpha=0.3)
    log.record(epochs+(i+1)/N,val_loss=loss,val_acc=acc,end='\r')
 

  log.report_avgs(epochs+1)

EPOCH: 1.000	trn_loss: -0.123	trn_acc: 0.928	val_loss: -0.181	val_acc: 0.970	(15.75s - 141.71s remaining)
EPOCH: 2.000	trn_loss: -0.183	trn_acc: 0.975	val_loss: -0.191	val_acc: 0.980	(31.27s - 125.08s remaining)
EPOCH: 3.000	trn_loss: -0.191	trn_acc: 0.983	val_loss: -0.196	val_acc: 0.985	(46.98s - 109.61s remaining)
EPOCH: 4.000	trn_loss: -0.195	trn_acc: 0.986	val_loss: -0.197	val_acc: 0.986	(62.67s - 94.00s remaining)
EPOCH: 5.000	trn_loss: -0.197	trn_acc: 0.988	val_loss: -0.200	val_acc: 0.988	(78.26s - 78.26s remaining)
EPOCH: 6.000	trn_loss: -0.199	trn_acc: 0.990	val_loss: -0.199	val_acc: 0.987	(94.06s - 62.71s remaining)
EPOCH: 7.000	trn_loss: -0.201	trn_acc: 0.991	val_loss: -0.201	val_acc: 0.989	(109.71s - 47.02s remaining)
EPOCH: 8.000	trn_loss: -0.202	trn_acc: 0.992	val_loss: -0.200	val_acc: 0.988	(125.25s - 31.31s remaining)
EPOCH: 9.000	trn_loss: -0.203	trn_acc: 0.993	val_loss: -0.200	val_acc: 0.989	(141.00s - 15.67s remaining)
EPOCH: 10.000	trn_loss: -0.203	trn_acc: 0.994	val